Project by:
- Jack Chen 4427737
- Joost Litjes 4540700
- Felicia Hung 7568479

In [1]:
import numpy as np
import pandas as pd

import os

import sklearn

from scipy import stats

import plotly.express as px 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

In [2]:
px.defaults.width = 600
px.defaults.height = 600

Task 1

In [3]:
db = pd.read_csv("online_shoppers_intention 1.csv")
# db = pd.read_csv("online_shoppers_intention 2.csv")

In [4]:
db.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


In [5]:
def exportImage(plot, name):
    pio.write_html(plot, os.path.join("plots", name + '.html'))
    
    # Change if you want to print plots !!
    # fig.show()

In [6]:
numeric_features = [
    "Administrative",
    "Informational",
    "ProductRelated",
    "Administrative_Duration",
    "Informational_Duration",
    "ProductRelated_Duration",
    "BounceRates",
    "PageValues",
    "SpecialDay",
]

categorical_features = [
    "TrafficType",
    "VisitorType",
    "OperatingSystems",
    "Browser",
    "Region",
    "Month",
    "Weekend",
    "Revenue",
]

In [7]:
db = db.astype({col: str for col in db.columns if col in categorical_features})
browser_13_df = db[db['Browser'] == "13"]
other_browsers_df = db[db['Browser'] != "13"]

In [8]:
fig = make_subplots(rows=len(numeric_features), cols=2,
                    subplot_titles=('Browser 13', 'Other Browsers'))

colors = ['blue', 'red']  # Define colors for the two databases

for j, feature_to_plot in enumerate(numeric_features):
    for i, database in enumerate([browser_13_df, other_browsers_df]):
        data = database[feature_to_plot]
        # Only show the feature name once in the subplot titles
        title = feature_to_plot if i == 0 else ''
        box_trace = go.Box(x=data, name=title, marker_color=colors[i], showlegend=False)
        fig.add_trace(box_trace, row=j+1, col=i+1)

fig.update_layout(height=len(numeric_features)*100, width=800, title_text="Comparing trends between Browser 13 and others for Numeric Features")
exportImage(fig, "Comparing trends between Browser 13 and others for Numeric Features")


In [9]:
fig = make_subplots(rows=len(categorical_features), cols=2,
                    subplot_titles=('Browser 13', 'Other Browsers'))

colors = ['blue', 'red']

for i, database in enumerate([browser_13_df, other_browsers_df]):
    for j, feature_to_plot in enumerate(categorical_features):
        data = database[feature_to_plot]
        unique_values = data.unique()
        box_trace = go.Bar(x=data.value_counts(), y=unique_values, text="", marker_color=colors[i], showlegend=False, orientation='h')
        fig.add_trace(box_trace, row=j+1, col=i+1)

for j, feature_to_plot in enumerate(categorical_features):
    fig.update_yaxes(title_text=feature_to_plot, row=j+1, col=1)

fig.update_layout(height=len(numeric_features)*150, width=1000, title_text="Comparing trends between Browser 13 and others for Categorical Features")
exportImage(fig, "Comparing trends between Browser 13 and others for Categorical Features")

Task 2

In [10]:
# Manual normalization function
def normalize_column(column):
    min_val = column.min()
    max_val = column.max()
    return (column - min_val) / (max_val - min_val)

for column in numeric_features:
    db[column] = normalize_column(db[column])


Task 3

In [11]:
# Convert categorical features to numerical using one-hot encoding
data_encoded = pd.get_dummies(db, columns=categorical_features, drop_first=True)

# Calculate the correlation matrix for dummified categorical features
correlation_matrix_categorical = list(data_encoded[
    data_encoded.columns.difference(numeric_features)
].columns)

correlation_matrix = data_encoded[numeric_features + correlation_matrix_categorical].corr()

numeric_features_indexes = [correlation_matrix.columns.get_loc(col) for col in numeric_features]
categorical_features_indexes = [correlation_matrix.columns.get_loc(col) for col in correlation_matrix_categorical]

data = correlation_matrix.iloc[numeric_features_indexes, numeric_features_indexes]
fig = px.imshow(
    data,
    labels=dict(x="Numeric Features", y="Numeric Features", color="Correlation"),
    title="Correlation Heatmap of Numerical Features",
)
fig.update_layout(height=1000, width=1000)
exportImage(fig, "Correlation Heatmap of Numerical Features")

data = correlation_matrix.iloc[categorical_features_indexes, categorical_features_indexes]
fig = px.imshow(
    data,
    labels=dict(x="Categorical Features", y="Categorical Features", color="Correlation"),
    title="Correlation Heatmap of Categorical Features",
)
fig.update_layout(height=1000, width=1000)
exportImage(fig, "Correlation Heatmap of Categorical Features")

data = correlation_matrix.iloc[numeric_features_indexes, categorical_features_indexes]
fig = px.imshow(
    data,
    labels=dict(x="Numeric Features", y="Categorical Features", color="Correlation"),
    title="Correlation Heatmap of Numerical vs Categorical Features",
)
fig.update_layout(height=1000, width=1000)
exportImage(fig, "Correlation Heatmap of Numerical vs Categorical Features")


In [12]:
fig = make_subplots(rows=len(numeric_features), cols=len(numeric_features))

for i, feature_to_plot_y in enumerate(numeric_features):
    for j, feature_to_plot_x in enumerate(numeric_features):
        trace = go.Scatter(x=db[feature_to_plot_x], y=db[feature_to_plot_y], text="", mode='markers', showlegend=True)
        fig.add_trace(trace, row=j+1, col=i+1)

# Add x and y labels to the subplots
for i, feature in enumerate(numeric_features):
    fig.update_xaxes(title_text=feature, row=len(numeric_features), col=i+1)
    fig.update_yaxes(title_text=feature, row=i+1, col=1)

fig.update_layout(height=len(numeric_features)*150, width=len(numeric_features)*150, title_text="Comparing trends between Browser 13 and others for Categorical Features")
exportImage(fig, "Scatter Plots")

In [13]:
from sklearn.decomposition import PCA
data = data_encoded

pca = PCA(n_components=2)
components = pca.fit_transform(data)
components_df = pd.DataFrame(components, columns=['PC1', 'PC2'])
fig = px.scatter(components_df, x='PC1', y='PC2')
fig.show()

exportImage(fig, "PCA")


In [14]:
from sklearn.cluster import DBSCAN, Birch, AffinityPropagation

def perform_clustering(data, clustering_algo, cluster_col_name, title, export_name, **kwargs):
    clusterer = clustering_algo(**kwargs)
    data[cluster_col_name] = clusterer.fit_predict(data)
    fig = px.scatter(data, x="PC1", y="PC2", color=cluster_col_name, title=title)
    exportImage(fig, export_name)
    return data

In [15]:
components_df = perform_clustering(components_df, DBSCAN, "DBSCAN_Cluster", "DBSCAN Clustering", "DBSCAN Clustering", eps=0.1, min_samples=5)

In [16]:
components_df = perform_clustering(components_df, Birch, "Birch_Cluster", "Birch Clustering", "Birch Clustering", threshold=0.5, branching_factor=100, n_clusters=4)

In [17]:
components_df = perform_clustering(components_df, AffinityPropagation, "AP_Cluster", "Affinity Propagation Clustering", "Affinity Propagation Clustering", damping=0.95, max_iter=50, convergence_iter=5)

Task 4

In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances

def silhouette_score(data, labels):
    distances = pairwise_distances(data)
    n = len(data)
    silhouette_values = np.zeros(n)

    for i in range(n):
        cluster_i = labels[i]
        a_i = np.mean(distances[i][labels == cluster_i])

        b_i = np.min([np.mean(distances[i][labels == cluster_j]) for cluster_j in np.unique(labels) if cluster_j != cluster_i])

        silhouette_values[i] = (b_i - a_i) / max(a_i, b_i)

    silhouette_avg = np.mean(silhouette_values)
    return silhouette_avg

from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score

In [ ]:
x = len(components_df)
data = components_df.iloc[:x, :2].to_numpy()

results = pd.DataFrame({
    'Model': ['AP', 'DBSCAN', 'Birch'],
    'Silhouette Score': [
        silhouette_score(data, components_df[:x]['AP_Cluster'].to_numpy()),
        silhouette_score(data, components_df[:x]['DBSCAN_Cluster'].to_numpy()),
        silhouette_score(data, components_df[:x]['Birch_Cluster'].to_numpy())
    ],
    'Davies-Bouldin Score': [
        davies_bouldin_score(data, components_df[:x]['AP_Cluster'].to_numpy()),
        davies_bouldin_score(data, components_df[:x]['DBSCAN_Cluster'].to_numpy()),
        davies_bouldin_score(data, components_df[:x]['Birch_Cluster'].to_numpy())
    ],
    'Calinski-Harabasz Score': [
        calinski_harabasz_score(data, components_df[:x]['AP_Cluster'].to_numpy()),
        calinski_harabasz_score(data, components_df[:x]['DBSCAN_Cluster'].to_numpy()),
        calinski_harabasz_score(data, components_df[:x]['Birch_Cluster'].to_numpy())
    ]
})

print(results)


    Model  Silhouette Score  Davies-Bouldin Score  Calinski-Harabasz Score
0      AP          0.307416              0.963267              4668.185766
1  DBSCAN          0.401499              2.608033              5524.599755
2   Birch          0.474736              0.641019             12667.449132


Task 5

In [ ]:
x = len(components_df)
data = components_df.iloc[:x, :2].to_numpy()
results = pd.DataFrame(columns=['eps', 'min_samples', 'silhouette_score'])

for eps in range(1, 7):
    eps_value = eps / 10.0
    for min_samples in range(3, 8):
        components_df = perform_clustering(
            components_df.iloc[:x, :3],
            DBSCAN,
            f"DBSCAN_eps_{eps_value}_min_samples_{min_samples}",
            f"DBSCAN_eps_{eps_value}_min_samples_{min_samples}",
            f"DBSCAN_eps_{eps_value}_min_samples_{min_samples}",
            eps=eps_value,
            min_samples=min_samples,
            metric='euclidean')
        
        sil_score = silhouette_score(data, components_df[:x][f"DBSCAN_eps_{eps_value}_min_samples_{min_samples}"].to_numpy())
        results = results.append({'eps': eps_value, 'min_samples': min_samples, 'silhouette_score': sil_score}, ignore_index=True)

fig = go.Figure(data=[go.Surface(
    z=results.pivot('eps', 'min_samples', 'silhouette_score').values,
    x=results['eps'].unique(),
    y=results['min_samples'].unique()
)])

fig.update_layout(
    title="Silhouette Score vs. DBSCAN Parameters",
    scene=dict(
        xaxis_title='eps',
        yaxis_title='min_samples',
        zaxis_title='silhouette_score'
    )
)
exportImage(fig, 'Silhouette Score vs. DBSCAN Parameters')
print(results)

C:\Users\jackc\AppData\Local\Temp\ipykernel_46072\2544114204.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\jackc\AppData\Local\Temp\ipykernel_46072\2544114204.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\jackc\AppData\Local\Temp\ipykernel_46072\2544114204.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\jackc\AppData\Local\Temp\ipykernel_46072\2544114204.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\jackc\AppData\Local\Temp\ipykernel_46072\2544114204.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

    eps  min_samples  silhouette_score
0   0.1          3.0         -0.028524
1   0.1          4.0          0.062798
2   0.1          5.0          0.401499
3   0.1          6.0          0.400667
4   0.1          7.0          0.390311
5   0.2          3.0         -0.050489
6   0.2          4.0          0.062798
7   0.2          5.0          0.401499
8   0.2          6.0          0.401499
9   0.2          7.0          0.401499
10  0.3          3.0         -0.033690
11  0.3          4.0         -0.033690
12  0.3          5.0          0.314311
13  0.3          6.0          0.401499
14  0.3          7.0          0.401499
15  0.4          3.0         -0.066716
16  0.4          4.0         -0.032518
17  0.4          5.0          0.138662
18  0.4          6.0          0.322838
19  0.4          7.0          0.401499
20  0.5          3.0         -0.022566
21  0.5          4.0         -0.022566
22  0.5          5.0         -0.022566
23  0.5          6.0          0.258125
24  0.5          7.0     

C:\Users\jackc\AppData\Local\Temp\ipykernel_46072\2544114204.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [ ]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))
    
def manhattan_distance(x1, x2):
    distance = 0
    for i in range(len(x1)):
        distance += abs(x1[i] - x2[i])
    return distance

def cosine_similarity_distance(x1, x2):
    dot_product = np.dot(x1, x2)
    norm_x1 = np.linalg.norm(x1)
    norm_x2 = np.linalg.norm(x2)
    
    if norm_x1 == 0 or norm_x2 == 0:
        return 1.0  # Handle division by zero
    
    return 1 - (dot_product / (norm_x1 * norm_x2))

In [ ]:
components_df = components_df.iloc[:x, :2]
x = len(components_df)
data = components_df.iloc[:x, :2].to_numpy()

components_df = perform_clustering(components_df, DBSCAN, "DBSCAN_Cluster_Custom_Euclidean_Distance", "Custom_Euclidean_Distance Clustering", "Custom_Euclidean_Distance Clustering", eps=0.1, min_samples=5, metric=euclidean_distance)
components_df = perform_clustering(components_df, DBSCAN, "DBSCAN_Cluster_Custom_Manhattan_Distance", "Custom_Manhattan_Distance Clustering", "Custom_Manhattan_Distance Clustering", eps=0.1, min_samples=5, metric=manhattan_distance)
components_df = perform_clustering(components_df, DBSCAN, "DBSCAN_Cluster_Custom_Cosine_Similarity_Distance", "Custom_Cosine_Similarity_Distance Clustering", "Custom_Cosine_Similarity_Distance Clustering", eps=0.1, min_samples=5, metric=manhattan_distance)

results = pd.DataFrame({
    'Model': ['euclidean', 'manhattan_distance', 'cosine_similarity_distance'],
    'Silhouette Score': [
        silhouette_score(data, components_df[:x]['DBSCAN_Cluster_Custom_Euclidean_Distance'].to_numpy()),
        silhouette_score(data, components_df[:x]['DBSCAN_Cluster_Custom_Manhattan_Distance'].to_numpy()),
        silhouette_score(data, components_df[:x]['DBSCAN_Cluster_Custom_Cosine_Similarity_Distance'].to_numpy())
    ],
    'Davies-Bouldin Score': [
        davies_bouldin_score(data, components_df[:x]['DBSCAN_Cluster_Custom_Euclidean_Distance'].to_numpy()),
        davies_bouldin_score(data, components_df[:x]['DBSCAN_Cluster_Custom_Manhattan_Distance'].to_numpy()),
        davies_bouldin_score(data, components_df[:x]['DBSCAN_Cluster_Custom_Cosine_Similarity_Distance'].to_numpy())
    ],
    'Calinski-Harabasz Score': [
        calinski_harabasz_score(data, components_df[:x]['DBSCAN_Cluster_Custom_Euclidean_Distance'].to_numpy()) / 1000,
        calinski_harabasz_score(data, components_df[:x]['DBSCAN_Cluster_Custom_Manhattan_Distance'].to_numpy()) / 1000,
        calinski_harabasz_score(data, components_df[:x]['DBSCAN_Cluster_Custom_Cosine_Similarity_Distance'].to_numpy()) / 1000
    ]
})

fig = px.bar(results, x='Model', y=['Silhouette Score', 'Davies-Bouldin Score', 'Calinski-Harabasz Score'],
             title='Cluster Evaluation Scores for Different Distance Metrics',
             labels={'value': 'Score'},
             height=400, width=600)
exportImage(fig, 'Clustering Evaluation Metrics')
print(results)

                        Model  Silhouette Score  Davies-Bouldin Score  \
0                   euclidean          0.401499              2.608033   
1          manhattan_distance         -0.043638              2.652118   
2  cosine_similarity_distance         -0.043638              2.652118   

   Calinski-Harabasz Score  
0              5524.599755  
1              1928.945902  
2              1928.945902  


In [ ]:
fig = px.bar(results, x='Model', y=['Silhouette Score', 'Davies-Bouldin Score', 'Calinski-Harabasz Score'],
             title='Cluster Evaluation Scores for Different Distance Metrics',
             labels={'value': 'Score'},
             height=400, width=600
             , barmode="group",)
exportImage(fig, 'Clustering Evaluation Metrics')
print(results)

                        Model  Silhouette Score  Davies-Bouldin Score  \
0                   euclidean               1.0                   0.0   
1          manhattan_distance               0.0                   1.0   
2  cosine_similarity_distance               0.0                   1.0   

   Calinski-Harabasz Score  
0                      1.0  
1                      0.0  
2                      0.0  


C:\Users\jackc\AppData\Local\Temp\ipykernel_46072\3766393741.py:7: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

